In [1]:
import pandas as pd
import re
from time import time
from sklearn.model_selection import train_test_split
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import nltk
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import pickle
from time import time
import os
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC

# 파이프라인으로 한 이유
- 폴드문제를 해결
- 여러개 알고리즘을 비교, 사용하기 위해
- 하이퍼 파라미터 튜닝을 위해

In [2]:
# 공백으로 단어분리
def tokenizer(text) :
    return text.split()

# 단어줄기
def tokenizer_porter(text) :
    return [porter.stem(word) for word in text.split()]

def tokenizer_stopwordsr(text) :
    # 띄어쓰기를 기준으로 분리한다.
    word_list = text.split()
    #단어 줄기 처리
    word_list2 = \
        [porter.stem(word) for word in word_list]
    #불용어 처리
    result = []
    for w in word_list2: 
        if w not in stop: 
            result.append(w)
    return result

def step3_word_tokenizer() :
    text = 'runners like running and thus they run'

    a1 = tokenizer(text)
    a2 = tokenizer_porter(text)
    print('a1 :', a1)
    print('a2 :', a2)

In [3]:
# csv 파일에서 데이터를 읽어온다.
df = pd.read_csv("./data/PN_dataset/dataset_1600.csv")
# 테스트, 학습데이터로 나눈다.
X_train = df.loc[:1120, 'review'].values
y_train = df.loc[:1120, 'PN'].values

X_test = df.loc[480:, 'review'].values
y_test = df.loc[480:, 'PN'].values

# 단어장을 만들어주는 객체 생성
tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)
# tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer_stopwordsr)
# 데이터를 학습하기 위한 객체
logistic = LogisticRegression(C=10.0, penalty='l2', random_state=0)

In [4]:
pipe = Pipeline( [ ('vect', tfidf),
                   ('clf', logistic)
                    ] )

In [5]:
##################################파라미터 의미#########################################
# vect__ngram_range : 
# vect__stop_words :
# vect__tokenizer : 
# vect__use_idf : 
# vect__norm : 
# clf__C : 

param_grid = [{"vect__ngram_range" : [(1,1)], "vect__stop_words" : [stopwords, None],
              "vect__tokenizer" : [tokenizer, tokenizer_porter],
               "clf__penalty" : ["l1", "l2"],
              "clf__C" : [1.0, 10.0, 100.0]},
             {"vect__ngram_range" : [(1,1)], "vect__stop_words" : [stopwords, None],
              "vect__tokenizer" : [tokenizer, tokenizer_porter],
              "vect__use_idf" : [False], "vect__norm" : [None],
              "clf__penalty" : ["l1", "l2"],
              "clf__C" : [1.0, 10.0, 100.0]}]

In [37]:
grid = GridSearchCV(pipe, param_grid, cv=5)

ValueError: Parameter values for parameter (clf__C) need to be a sequence(but not a string) or np.ndarray.

In [41]:
grid.fit(X_train, y_train)

C:\Users\admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: 'WordListCorpusReader' object is not iterable

  FitFailedWarning)
C:\Users\admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
C:\Users\admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
C:\Users\admin\anaconda3\lib\site-packages\sklearn\model_selection\

C:\Users\admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
NameError: name 'porter' is not defined

  FitFailedWarning)
C:\Users\admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: 'WordListCorpusReader' object is not iterable

  FitFailedWarning)
C:\Users\admin\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs 

In [17]:
# 최고 점수
grid.best_score_

0.8688412698412697

In [18]:
# 최적 파라미터
grid.best_params_

{'clf__C': 10.0,
 'clf__penalty': 'l2',
 'vect__ngram_range': (1, 1),
 'vect__stop_words': None,
 'vect__tokenizer': <function __main__.tokenizer(text)>}

In [40]:
X_train

array(["So people just go nuts for this place and I'm not entirely sure why. We tried this location out last night for a quick and easy Friday night dinner while we were out and about running errands. We got there before the crowd, and our overall impression was that the location is nice, but a little hidden, but that is no fault of the restaurant. \r\n\r\nAll things considered, the food was pretty decent. We did not get the wings, as we are trying to be a little bit more healthy. We split the salmon stuffed avocado and it was actually pretty delicious. We also split an order of the Korean tacos. Here is where they lost stars. \r\n\r\nThe service was horrible. They had one girl sitting at the front door on her phone pretty much the whole time. Two other ladies were servers, and seemed to hit every table but ours. Even as the restaurant filled up we still expected someone to come refill our water, ask us if we wanted another beer, or at least come take our plates away when we were done 

In [19]:
# 예측 평가
grid.score(X_test,y_test)

0.9419642857142857

In [20]:
from ml_utils.regression import adjusted_r2

In [30]:
adjusted_r2(grid, X_test, y_test)

ValueError: not enough values to unpack (expected 2, got 1)

# 단일 모델을 만들어보자

In [71]:
from sklearn.model_selection import train_test_split

In [99]:
df = pd.read_csv("./data/PN_dataset/dataset_1600.csv")
# 테스트, 학습데이터로 나눈다.
# X_train = df.loc[:1120, 'review'].values
# y_test = df.loc[:1120, 'PN'].values

# X_test = df.loc[480:, 'review'].values
# y_test = df.loc[480:, 'PN'].values

# # 단어장을 만들어주는 객체 생성
# tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer, ngram_range=(1, 1), stop_words=None)
# # tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer_stopwordsr)

- X, Y를 나눠준다

In [129]:
X = df['review'].values
Y = df['PN'].values

- X를 벡터로 바꿔준다

In [130]:
tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer, ngram_range=(1, 1), stop_words=None)

In [131]:
df_train = tfidf.fit_transform(X)

- 바꿔준걸 Train,test로 나눈다

In [132]:
X_train, X_test, Y_train, Y_test = train_test_split(df_train, Y, test_size=0.33, random_state=3)

In [133]:
print(X_train.shape)

(1072, 15491)


In [134]:
print(X_test.shape)

(528, 15491)


- 알고리즘에 파라미터 적용 후 선언해준다

In [138]:
from sklearn.linear_model import LogisticRegressionCV

In [144]:
logistic = LogisticRegressionCV(Cs=10, penalty='l2', random_state=0, cv=5).fit(X_train,Y_train)

- 학습시킨다

In [145]:
modle=logistic.fit(X_train, Y_train)

- 예측

In [146]:
predict = logistic.predict(X_test)

In [147]:
logistic.score(X_test, Y_test)

0.8636363636363636

In [148]:
modle.summary

AttributeError: 'LogisticRegressionCV' object has no attribute 'summary'

In [149]:
from ml_utils.regression import adjusted_r2

In [150]:
adjusted_r2(modle, X_test, Y_test)

1.0192273957497995